In [2]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import sys
# import snowflake.connector
import ast
# from pandas.io.json import json_normalize
import json
from datetime import date
from datetime import timedelta
import os

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns',200)
from IPython.display import display
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)

import yfinance as yf

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import yfinance as yf
import pandas as pd
import time

from scipy.optimize import minimize

import seaborn as sns

from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage,dendrogram

In [3]:
nifty_100 = pd.read_csv(r"C:\Users\Akash Dixit\Downloads\ind_nifty100list (1).csv")
indian_tickers = []

for i in nifty_100['Symbol'].unique():
    temp = f"{i}.NS"
    indian_tickers.append(temp)

results = []

for symbol in indian_tickers:
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        
        dct = {
            'Ticker': symbol,
            'Company': info.get('longName'),
            'Sector': info.get('sector'),
            'Industry': info.get('industry'),
            'MarketCap': info.get('marketCap')
        }
        
        results.append(dct)
        time.sleep(0.5)  
        
    except Exception as e:
        print(f"Failed to fetch data for {symbol}: {e}")


company_details = pd.DataFrame(results)


company_details['MarketCap_Cr'] = company_details['MarketCap'] / 1e7

print(company_details.head())




          Ticker                                        Company       Sector  \
0         ABB.NS                              ABB India Limited  Industrials   
1  ADANIENSOL.NS                 Adani Energy Solutions Limited    Utilities   
2    ADANIENT.NS                      Adani Enterprises Limited       Energy   
3  ADANIGREEN.NS                     Adani Green Energy Limited    Utilities   
4  ADANIPORTS.NS  Adani Ports and Special Economic Zone Limited  Industrials   

                                  Industry      MarketCap   MarketCap_Cr  
0           Specialty Industrial Machinery  1073995317248  107399.531725  
1  Utilities - Independent Power Producers   994900115456   99490.011546  
2                             Thermal Coal  2757479366656  275747.936666  
3                    Utilities - Renewable  1603365109760  160336.510976  
4                          Marine Shipping  2958876213248  295887.621325  


In [4]:
class Price_Generator:

    def __init__(self,ticker,company_details):
        self.ticker = ticker
        self.company_details = company_details
        self.data = None

        if self.company_details is not None:
            self.company_details['company'] = self.company_details['Ticker'].str.replace(".NS", "", regex=False)


    def fetch_price(self,start_date,end_date):
            self.data = yf.download(self.ticker,start=start_date,end=end_date)
            self.data.columns = [f"{ticker.split('.')[0]}_{typ}" for typ, ticker in self.data.columns]
            return self.data

    def price_generation(self):
        if self.data is None:
            raise ValueError('Please run first fetch_price() before generating Price.')

        df = self.data.sort_index().reset_index()
        price_cols = [x for x in df.columns if 'Close' in x]
        
        melted = df.melt(id_vars='Date',
                         value_vars=price_cols,
                         var_name='company',
                         value_name='price')
        
        melted['company'] = melted['company'].str.replace("_Close","",regex=False)
        company_details['company'] = company_details['Ticker'].str.replace(".NS","",regex=False)
    
        melted = pd.merge(melted,
                         company_details,
                         on='company',
                         how='left')
        
        # melted = melted[melted['price'].notna()]
        
        # Daily Return
        melted['daily_return'] = (
            
            melted.sort_values(by=['company','Date'])
            .groupby(['company'])['price']
            .transform('pct_change')
        )
        melted['daily_return'] = melted['daily_return'].fillna(0)
        print('Generated Daily Return')

        # Cummulaive Return
        melted['Cummulative_return'] = (
            
            melted.sort_values(by=['company','Date'])
            .groupby(['company'])['daily_return']
            .transform(lambda x:(1+x).cumprod()-1)
        )
        print('Generatd Cummulative Return')
        
        
        return melted.drop(columns=['Ticker', 'Company'], errors='ignore')
            

processor = Price_Generator(ticker=indian_tickers,company_details=company_details)
#  Select Date
processor.fetch_price(start_date='2021-06-01',end_date='2025-07-31')
final_price_df = processor.price_generation()

[*********************100%***********************]  100 of 100 completed


Generated Daily Return
Generatd Cummulative Return


In [5]:
daily_return = final_price_df.pivot(index='Date',
                                         columns='company',
                                         values='daily_return')

nan_companies = ['ADANIENSOL','BAJAJHFL','ETERNAL','HYUNDAI','JIOFIN','SWIGGY','UNITDSPR']
daily_return = daily_return.drop(columns=nan_companies, errors='ignore')

### Heirarchal Risk Parity

In [6]:
# def get_hrp_weights(df):
#     nan_companies = ['ADANIENSOL','BAJAJHFL','ETERNAL','HYUNDAI','JIOFIN','SWIGGY','UNITDSPR']
#     df = df.drop(columns=nan_companies, errors='ignore')
#     cov = df.cov()
#     corr = df.corr()
    
#     def get_distance(corr_):
#         return np.sqrt(0.5*(1-corr_))

#     def get_quasi_diagonal(matrix):
#         temp_matrix = matrix.copy()
#         temp_matrix = matrix.astype(int)
#         root_idx = pd.Series([matrix[-1,0],matrix[-1,1]])
#         num_asstes = matrix[-1,-1]

#         while root_idx.max()>= num_asstes:
#             root_idx.index = range(0,root_idx.shape[0]*2,2)
#             dx0 = root_idx[root_idx>=num_asstes]
#             for i in dx0.index:
#                 val = root_idx[i]
#                 root_idx[i] = temp_linkage_matrix[int(val-num_asstes),0]
                
#                 root_idx = pd.concat([root_idx.iloc[:i+1],
#                                      pd.Series(temp_linkage_matrix[int(val-num_asstes),1]),
#                                      root_idx.iloc[i+1:]
#                                      ])
#                 lst = root_idx.tolist()
#                 lst = [int(x) for x in lst]
#         return lst

#     def get_variance(cov_,items):
#         cov_sub = cov_.loc[items,items]
#         w_ = np.linalg.inv(cov_sub).sum(axis=1)
#         w_ /= w_.sum()
#         return np.dot(np.dot(w_,cov_sub),w_)

#     def recursive_bisection(tickers):
#         ticks = [tickers]
#         weight = pd.Series(1,index=tickers)
        
#         while len(ticks)>0:
#             ticks = [i[j:k] for i in ticks for j,k in ((0,len(i)//2),(len(i)//2,len(i))) if len(i)>1]        
#             for i in range(0,len(ticks),2):
#                 c0 = ticks[i]
#                 c1 = ticks[i+1]
#                 var0 = get_variance(cov,c0)
#                 var1 = get_variance(cov,c1)
#                 alpha = 1 - (var0/(var0+var1))
#                 weight[c0] *= alpha
#                 weight[c1] *= 1-alpha
#         return weight

#     # HRP PROCESS
#     distance = get_distance(corr)
#     linkage_matrix = linkage(squareform(distance),method='single')
#     quasi_diagonal = get_quasi_diagonal(linkage_matrix)
#     sorted_tickers = cov.index[quasi_diagonal]
#     hrp_weights = recursive_bisection(sorted_tickers)

#     hrp_weights = hrp_weights/hrp_weights.sum()
#     return hrp_weights


import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import squareform

def get_hrp_weights(price_df):
    """
    Calculate Hierarchical Risk Parity (HRP) weights from price data.
    
    Parameters:
        price_df (pd.DataFrame): Asset prices (columns = tickers, index = dates)
    
    Returns:
        pd.Series: Asset weights
    """

    nan_companies = ['ADANIENSOL','BAJAJHFL','ETERNAL','HYUNDAI','JIOFIN','SWIGGY','UNITDSPR']
    price_df = price_df.drop(columns=nan_companies, errors='ignore')
    cov = price_df.cov()
    corr = price_df.corr()

    def distance(corr_):
        return np.sqrt(0.5 * (1 - corr_))

    def quasi_diagonalization(matrix):
        temp_matrix = matrix.astype(int)
        root_node = pd.Series([temp_matrix[-1, 0], temp_matrix[-1, 1]])
        num_assets = temp_matrix[-1, -1]
        
        while root_node.max() >= num_assets:
            root_node.index = range(0, root_node.shape[0] * 2, 2)
            dx0 = root_node[root_node >= num_assets]
            for i in dx0.index:
                root_i = root_node[i]
                root_node[i] = temp_matrix[int(root_i - num_assets), 0]
                root_node = pd.concat([
                    root_node.iloc[:i+1],
                    pd.Series(temp_matrix[int(root_i - num_assets), 1]),
                    root_node.iloc[i+1:]
                ])
        return root_node.tolist()

    def get_variance(cov_, items):
        cov_sub = cov_.loc[items, items]
        w_ = np.linalg.inv(cov_sub).sum(axis=1)
        w_ /= w_.sum()
        return np.dot(np.dot(w_, cov_sub), w_)

    def recursive_bisection(ticker_list):
        tick = [ticker_list]
        w = pd.Series(1, index=ticker_list)
        
        while len(tick) > 0:
            tick = [i[j:k] for i in tick for j, k in ((0, len(i)//2), (len(i)//2, len(i))) if len(i) > 1]
            for i in range(0, len(tick), 2):
                c0 = tick[i]
                c1 = tick[i+1]
                var0 = get_variance(cov, c0)
                var1 = get_variance(cov, c1)
                alpha = 1 - (var0 / (var0 + var1))
                w[c0] *= alpha
                w[c1] *= 1 - alpha
        return w

    # HRP process
    dist_matrix = distance(corr)
    linkage_matrix = linkage(squareform(dist_matrix), method='single')
    quasi_diag = quasi_diagonalization(linkage_matrix)
    sorted_tickers = cov.index[quasi_diag]
    hrp_weight = recursive_bisection(sorted_tickers)
    
    hrp_weight = hrp_weight / hrp_weight.sum()  # Normalize to sum = 1
    return hrp_weight


### Mean Variance Optimization

In [7]:
mean = daily_return.mean()
cov = daily_return.cov()
var = daily_return.var()

In [8]:
def get_mvo_weights(df):
    
    nan_companies = ['ADANIENSOL','BAJAJHFL','ETERNAL','HYUNDAI','JIOFIN','SWIGGY','UNITDSPR']
    df = df.drop(columns=nan_companies, errors='ignore')
    mean = df.mean()
    cov = df.cov()
    var = df.var()

    def portfolio_volatility(w,cov):
        return np.sqrt(w.T @ cov @ w)

    def portfolio_return(w,mean):
        return w @ mean

    assets = cov.columns
    num_assets = len(assets)
    initial_guess = np.random.dirichlet(np.ones(num_assets),size=1)[0]
    bound = tuple((0,0.15) for _ in range(num_assets))
    returns = 0.001
    constraints = (
        {'type':'eq','fun':lambda w:np.sum(w)-1},
        {'type':'eq','fun':lambda w:portfolio_return(w,mean)-returns}
    )
    
    results = minimize(
        portfolio_volatility,
        x0=initial_guess,
        constraints=constraints,
        bounds=bound,
        method='SLSQP',
        args=(cov,)
    )
    
    optimal_weights = results.x
    weights = pd.Series(optimal_weights,index=assets)
    weights = round(weights,7)
    return dict(weights)

    

#### Equal Weights

In [9]:
def get_equal_weights(df):
    
    nan_companies = ['ADANIENSOL','BAJAJHFL','ETERNAL','HYUNDAI','JIOFIN','SWIGGY','UNITDSPR']
    df = df.drop(columns=nan_companies, errors='ignore')
    mean = df.mean()
    cov = df.cov()
    assets = cov.columns.tolist()
    num_assets = len(assets)
    equal_w = num_assets * [1/num_assets]
    equal_weights = dict(zip(assets,equal_w))
    return equal_weights

#### Risk Parity

In [10]:
def get_riskparity_weights(df):

    nan_companies = ['ADANIENSOL','BAJAJHFL','ETERNAL','HYUNDAI','JIOFIN','SWIGGY','UNITDSPR']
    df = df.drop(columns=nan_companies, errors='ignore')
    corr = df.corr()
    mean = df.mean()
    cov = df.cov()
    var = df.var()
    assets = cov.columns.tolist()

    def calculate_portfolio_volatility(w,cov):
        return np.sqrt(w.T @ cov @ w)

    def calculate_marginal_risk(w,cov):
        portfolio_vol = calculate_portfolio_volatility(w,cov)
        marginal_contribution = cov @ w
        risk_contribution = marginal_contribution * w
        return risk_contribution/portfolio_vol
    
    def get_risk_parity(w,cov):
        risk_contrib = calculate_marginal_risk(w,cov)
        mean_risk = risk_contrib.mean()
        return np.sum((risk_contrib-mean_risk)**2)
    
    def ret_risk_parity_weight(cov):
        num_assets = cov.shape[0]
        # init_guess = np.ones(num_assets)/num_assets
        init_guess = np.random.dirichlet(np.ones(num_assets), size=1)[0]
        bound = [(0,0.30) for _ in range(num_assets)]
        constraints = ({'type':'eq','fun':lambda w:np.sum(w)-1})
        
        result = minimize(fun=get_risk_parity,
                         x0=init_guess,
                         args=(cov,),
                         bounds=bound,
                         method='SLSQP',
                         constraints=constraints)
        if result.success:
            return result.x
        else:
            print('Process Failed')
            
    rp_weights = ret_risk_parity_weight(cov)
    return dict(zip(cov.index, rp_weights))


In [11]:
def portfolio_generation(df,invest_amount):

    initial_price = (
        df.sort_values(by=['company','Date'])
        .groupby('company')
        .first()
    ).reset_index()

    for weight in [x for x in initial_price.columns if 'weights' in x]:
        if 'hrp' in weight.lower():
            initial_price['HRP_capital_allocated'] = initial_price[weight] * invest_amount
            initial_price['HRP_unit_allocated'] = (initial_price['HRP_capital_allocated'] / initial_price['price'])
        elif 'mean_var' in weight.lower():
            initial_price['MeanVar_Opt_capital_allocated'] =  initial_price[weight] * invest_amount
            initial_price['MeanVar_Opt_unit_allocated'] = (initial_price['MeanVar_Opt_capital_allocated'] / initial_price['price'])
        elif 'equal' in weight.lower() :
            initial_price['Equal_Weight_capital_allocated'] = initial_price[weight] * invest_amount
            initial_price['Equal_Weight_unit_allocated'] = (initial_price['Equal_Weight_capital_allocated'] / initial_price['price'])
        elif 'risk_parity' in weight.lower() :
            initial_price['Risk_Parity_capital_allocated'] = initial_price[weight] * invest_amount
            initial_price['RiskParity_unit_allocated'] = (initial_price['Risk_Parity_capital_allocated'] / initial_price['price'])
        else:
            pass

    unit_cols = ['company'] + [x for x in initial_price.columns if 'unit_allocated' in x]
    unit_df = initial_price[unit_cols]

    portfolio_price = pd.merge(df,
                          unit_df,
                          on='company',
                          how='left')

    
    
    for i in [x for x in portfolio_price.columns if 'unit_allocated' in x]:
        lst = i.split('_unit')
        # display(lst[0])
        portfolio_price[f"{lst[0]}_position_value"] = portfolio_price[i] * portfolio_price['price']
    
    overall_portfolio = portfolio_price.groupby(['Date'])[[x for x in portfolio_price.columns if '_position_value' in x]].sum().reset_index()
    
    for value in  [x for x in overall_portfolio.columns if '_position_value' in x]:
        # print(value)
        overall_portfolio[f"{(value.split('_'))[0]}_daily_return"] = overall_portfolio[value].pct_change()
        overall_portfolio[f"{(value.split('_'))[0]}_daily_return"] = overall_portfolio[f"{(value.split('_'))[0]}_daily_return"].fillna(0)
        overall_portfolio[f"{(value.split('_'))[0]}_rolling_volatility_21D"] = overall_portfolio[f"{(value.split('_'))[0]}_daily_return"].rolling(window=21).std()
    
        
        overall_portfolio[f"{(value.split('_'))[0]}_cummulative_return"] = (1 + overall_portfolio[f"{(value.split('_'))[0]}_daily_return"]
                                                                           ).cumprod() - 1
    return overall_portfolio,portfolio_price

    
    

In [12]:
def sharpe_calculation(df):
    cols = [x for x in df.columns if 'daily_return' in x or 'rolling_volatility_21D' in x]
    uniq = [x for x in df.columns if 'daily_return' in x ]
    dct = {}

    for i in uniq:
        temp = i.replace("_daily_return","")
        returns = None
        vol = None
        
        for j in cols:
            if temp in j:
                if 'daily_return' in j:
                    returns =  (df[j].mean()) * 252
                elif 'rolling_volatility_21D' in j:
                    vol = (df[j].mean()) * (252 ** 0.5)
                else:
                    pass

        if returns is not None and vol is not None:
            sharpe = returns/vol
            dct[temp] = sharpe
            dff = pd.DataFrame([dct],index=['sharpe']).T
            # print(f"{'Portfolio':<12}{'Sharpe Ratio'}")
            # print(temp, round(sharpe, 3))
        else:
            print(f"Missing data for {temp}")
    return pd.DataFrame.from_dict(dct, orient='index', columns=['Sharpe'])
        
def calculate_drawdown(df):
    frame = pd.DataFrame(index=df.index)
    for i in  [x for x in df.columns if 'cummulative_return' in x ]:
        df = df.sort_values(by='Date')
        print(i)
        temp = df.copy()
        name = i.split('_')[0]
        temp[i] = (1 + temp[i])
        cummax = temp[i].cummax()
        cum_retrun = temp[i]
        drawdown = ((cum_retrun) - (cummax))/cummax
        frame[f"{name}_drawdown"] = drawdown
    return frame
        

def drawdown_details(df):
    result = []
    for i in [x for x in df.columns if 'drawdown' in x]:
        df = df.sort_values(by='Date')
        max_dd_date = df.loc[df[i].idxmin(), 'Date']
        max_dd = df[i].min()
        max_dd = df[df.index == df[i].idxmin()][i].iloc[0]
        after_max_dd = df[df['Date']>max_dd_date]
        min_recovery_date = after_max_dd[after_max_dd[i]==0]['Date'].min()
        recovery_days = (min_recovery_date - max_dd_date).days
        
        frame = result.append({
            'Portfolio':i,
            'Max Drawdown': max_dd,
            'Date of Worst Drawdown':max_dd_date,
            'Days of Recovery':recovery_days
            
        })
    return pd.DataFrame(result)
        
        
    

def calculate_calmar_ratio(df):
    lst = []
    for i in [x for x in df.columns if 'daily_return' in x]:
        df = df.sort_values(by='Date')
        name = i.split('_')[0]
        daily_return = ((1+df[i].mean())**252)-1
        max_drawdown = df[f"{name}_drawdown"].min()
        calmar_ratio = daily_return/abs(max_drawdown)
        lst.append({
            'Name':name,
            'Calmar Ratio':calmar_ratio
        })
    return pd.DataFrame(lst)


def calcualte_var(df):
    # var_df = pd.DataFrame()
    dct = {}
    confidence_level = 0.05
    for i in [x for x in df.columns if 'daily_return' in x]:
        name = i.replace('_daily_return','')
        dct[name] = df[i].quantile(confidence_level)
        temp_dff = pd.DataFrame([dct]).T
        temp_dff.rename(columns={0:'Var'},inplace=True)
    return temp_dff
        
        

In [13]:
final_price_df_1 = final_price_df.copy()
final_price_df_1 = final_price_df_1[~(final_price_df_1['company'].isin(['ADANIENSOL','BAJAJHFL','ETERNAL','HYUNDAI','JIOFIN','SWIGGY','UNITDSPR'
                                                                        ,'LICI']))]

In [14]:
start_date = final_price_df_1['Date'].min()
end_date = start_date + pd.DateOffset(months=12)
test_date = end_date + pd.DateOffset(months=1)
final_date = final_price_df_1['Date'].max()
test_start_date = end_date + pd.Timedelta(days=1)
test_end_date = test_start_date + pd.DateOffset(months=6)

In [15]:
end_date

Timestamp('2022-06-01 00:00:00')

In [16]:
optimizers = {
    'hrp_weights':get_hrp_weights,
    'mean_var_opt_weights':get_mvo_weights,
    'equal_weights':get_equal_weights,
    'risk_parity_weights':get_riskparity_weights
}

name_mapping = {
    'HRP':'hrp_weights',
    'MeanVar':'mean_var_opt_weights',
    'Equal':'equal_weights',
    'RiskParity':'risk_parity_weights'
}

start_date = final_price_df_1['Date'].min()
end_date = start_date + pd.DateOffset(months=12)
test_date = end_date + pd.DateOffset(months=1)
final_date = final_price_df_1['Date'].max()
test_start_date = end_date + pd.Timedelta(days=1)
test_end_date = test_start_date + pd.DateOffset(months=6)
train_data = final_price_df_1[(final_price_df_1['Date']>=start_date)&(final_price_df_1['Date']<=end_date)]
train_data_daily_returns = train_data.pivot(index='Date',
                                           columns='company',
                                           values='daily_return')

nan_companies = ['ADANIENSOL','BAJAJHFL','ETERNAL','HYUNDAI','JIOFIN','SWIGGY','UNITDSPR']
train_data_daily_returns = train_data_daily_returns.drop(columns=nan_companies, errors='ignore')



dct = {}

for name,opt_fun in optimizers.items():
    dct[name] = opt_fun(train_data_daily_returns)
    
weights_df = pd.DataFrame(dct,index=daily_return.columns)

for i in weights_df.columns:
    weights_df[i] = round(weights_df[i],6)

weights_df = weights_df.reset_index()

In [17]:
train_data_daily_returns

company,ABB,ADANIENT,ADANIGREEN,ADANIPORTS,ADANIPOWER,AMBUJACEM,APOLLOHOSP,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJAJFINSV,BAJAJHLDNG,BAJFINANCE,BANKBARODA,BEL,BHARTIARTL,BOSCHLTD,BPCL,BRITANNIA,CANBK,CGPOWER,CHOLAFIN,CIPLA,COALINDIA,DABUR,DIVISLAB,DLF,DMART,DRREDDY,EICHERMOT,GAIL,GODREJCP,GRASIM,HAL,HAVELLS,HCLTECH,HDFCBANK,HDFCLIFE,HEROMOTOCO,HINDALCO,HINDUNILVR,ICICIBANK,ICICIGI,ICICIPRULI,INDHOTEL,INDIGO,INDUSINDBK,INFY,IOC,IRFC,ITC,JINDALSTEL,JSWENERGY,JSWSTEEL,KOTAKBANK,LODHA,LT,LTIM,M&M,MARUTI,MOTHERSON,NAUKRI,NESTLEIND,NTPC,ONGC,PFC,PIDILITIND,PNB,POWERGRID,RECLTD,RELIANCE,SBILIFE,SBIN,SHREECEM,SHRIRAMFIN,SIEMENS,SUNPHARMA,TATACONSUM,TATAMOTORS,TATAPOWER,TATASTEEL,TCS,TECHM,TITAN,TORNTPHARM,TRENT,TVSMOTOR,ULTRACEMCO,VBL,VEDL,WIPRO,ZYDUSLIFE
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-06-02,0.004311,0.092145,0.022619,0.016662,0.011012,0.021283,0.010433,-0.009314,-0.010937,0.013234,-0.000533,-0.000613,0.004123,0.023314,0.023137,-0.006380,0.012854,0.008070,0.008269,0.029865,0.010125,0.021350,0.008717,0.010162,-0.008938,-0.001386,0.012530,0.007028,-0.000781,0.004483,-0.009972,0.001826,-0.001592,0.005186,0.016757,-0.007310,-0.005094,0.016009,-0.001394,0.010387,0.000212,-0.002692,0.000205,0.025543,-0.008091,0.010855,0.017771,-0.006164,0.008644,-0.004237,-0.029036,0.005801,0.038492,0.016631,-0.007735,-0.013252,-0.002441,-0.013417,-0.000124,0.013193,0.134008,0.022539,-0.005913,0.008621,0.001275,0.020562,-0.001282,0.056423,0.013592,0.019594,0.014915,0.018731,0.010866,0.009801,0.013286,0.014738,0.009537,0.011640,0.015404,0.032967,0.021028,-0.007469,-0.013149,-0.003582,0.003357,0.015413,0.006447,-0.001083,0.012119,0.025909,0.000368,0.012834
2021-06-03,-0.005057,0.023501,-0.009925,0.017006,0.045124,0.008608,0.046447,0.010900,0.018521,-0.005052,-0.000317,0.007107,0.015985,0.007389,0.005654,0.004816,0.016291,-0.001580,0.006043,-0.003358,0.005307,0.022161,-0.006756,-0.000335,0.016064,0.004259,0.023204,0.041256,-0.005610,0.033610,0.013850,0.016165,0.011440,0.011620,0.024697,-0.004556,0.011004,0.003773,0.012027,0.001003,0.001887,0.002931,0.005248,0.006359,0.004079,-0.000028,-0.021644,0.007979,0.014434,0.004255,0.000239,-0.003510,0.040504,0.011401,0.017441,0.007605,0.027055,-0.000247,-0.005148,0.003452,-0.068934,0.011540,0.008265,0.005848,0.040340,-0.006990,0.007750,-0.006818,-0.005056,-0.001012,0.003816,0.002489,0.014772,0.016285,-0.008636,0.024430,-0.002878,0.009576,0.008204,0.005088,-0.001824,0.003770,-0.002557,0.066532,-0.007084,0.001875,0.013379,0.007203,0.008882,0.008176,-0.007274,0.011157
2021-06-04,-0.024498,0.074443,-0.007499,0.010360,0.049628,-0.005689,-0.013282,-0.003560,-0.011457,-0.005452,0.028410,-0.000368,0.015581,0.000611,0.010582,0.001222,0.005453,0.009179,-0.001988,-0.013170,-0.008211,0.001843,-0.001951,0.027172,0.006010,0.006947,-0.002352,0.001347,-0.005215,-0.008364,0.005899,-0.013652,0.030506,0.012725,-0.014685,-0.003087,-0.012890,0.003611,0.013313,-0.010396,-0.008971,-0.011611,-0.018511,0.015008,0.010340,-0.005752,0.003530,-0.002878,0.019119,0.004237,-0.001435,0.003020,-0.007712,0.009943,-0.001654,0.008030,0.017672,0.004977,0.003179,0.000721

In [18]:
# start_date = final_price_df_1['Date'].min()
start_date = pd.to_datetime('2022-06-01',format="%Y-%m-%d")
end_date = start_date + pd.DateOffset(months=6)
test_date = end_date + pd.DateOffset(months=1)
final_date = final_price_df_1['Date'].max()
test_start_date = end_date + pd.Timedelta(days=1)
test_end_date = test_start_date + pd.DateOffset(months=6)


# For first year apply HRP Weights
initial_strategy = final_price_df_1[(final_price_df_1['Date']>=start_date)&(final_price_df_1['Date']<=test_end_date)]

initial_strategy = pd.merge(initial_strategy,weights_df[['company','hrp_weights']],on='company',how='left')
initial_strategy_portfolio_genreation,initial_strategy_info = portfolio_generation(initial_strategy,100000)
renamed_cols = []
for i in [x for x in initial_strategy_portfolio_genreation.columns if 'HRP' in x]:
    renamed_cols.append(i.replace('HRP_',""))
initial_strategy_portfolio_genreation['strategy_used'] = 'HRP'
initial_strategy_portfolio_genreation.columns = ['Date'] + renamed_cols + ['strategy_used']


In [19]:
initial_strategy_portfolio_genreation

,Date,position_value,daily_return,rolling_volatility_21D,cummulative_return,strategy_used
0,2022-06-01,100000.000000,0.000000,NaN,0.000000,HRP
1,2022-06-02,100314.304169,0.003143,NaN,0.003143,HRP
2,2022-06-03,99227.940842,-0.010830,NaN,-0.007721,HRP
3,2022-06-06,98998.733907,-0.002310,NaN,-0.010013,HRP
4,2022-06-07,98233.286611,-0.007732,NaN,-0.017667,HRP
...,...,...,...,...,...,...
245,2023-05-29,124457.122280,0.006011,0.004529,0.244571,HRP
246,2023-05-30,124465.579707,0.000068,0.004268,0.244656,HRP
247,2023-05-31,124992.789738,0.004236,0.004128,0.249928,HRP
248,2023-06-01,124884.116733,-0.000869,0.004176,0.248841,HRP


In [122]:
final_df = initial_strategy_portfolio_genreation.copy()
strat_df_portfolio_generation = None
# final_df = initial_strategy_portfolio_genreation.copy()
start_date = pd.to_datetime('2022-06-01',format="%Y-%m-%d")
final_date = final_price_df_1['Date'].max()
strategies = []
from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()

while start_date < final_date:
    print(start_date)
    end_date = start_date + pd.DateOffset(months=6)
    test_date = end_date + pd.DateOffset(months=1)

    # Data for Weight Selection
    train_data = final_price_df_1[(final_price_df_1['Date']>=start_date)&(final_price_df_1['Date']<=end_date)]
    train_data_daily_returns = train_data.pivot(index='Date',columns='company',values='daily_return')

    nan_companies = ['ADANIENSOL','BAJAJHFL','ETERNAL','HYUNDAI','JIOFIN','SWIGGY','UNITDSPR']
    train_data_daily_returns = train_data_daily_returns.drop(columns=nan_companies, errors='ignore')

    # Weight Genration
    dct = {}
    for name,opt_fun in optimizers.items():
        dct[name] = opt_fun(train_data_daily_returns)
    weights_df = pd.DataFrame(dct,index=daily_return.columns) 
    for i in weights_df.columns:
        weights_df[i] = round(weights_df[i],6)    
    weights_df = weights_df.reset_index()

    train_data = pd.merge(train_data,weights_df,on='company',how='left')
    
    train_portfolio_return,train_temp = portfolio_generation(train_data,100000)
    drawdown_df = calculate_drawdown(train_portfolio_return)
    train_portfolio_return = pd.concat([train_portfolio_return,drawdown_df],axis=1)

    # Ratio Generation
    train_sharpe = sharpe_calculation(train_portfolio_return)
    train_calmar = calculate_calmar_ratio(train_portfolio_return)
    train_calmar.set_index('Name',inplace=True)
    train_var = calcualte_var(train_portfolio_return)
    drawdown_details_df = drawdown_details(train_portfolio_return)
    drawdown_details_df['Portfolio'] = drawdown_details_df['Portfolio'].str.replace('_drawdown',"")
    drawdown_details_df.set_index('Portfolio',inplace=True)
    metric_train = pd.concat([train_sharpe,train_calmar,drawdown_details_df[['Max Drawdown']],train_var],axis=1)

    # Data for weight Testing
    test_start_date = end_date + pd.Timedelta(days=1)
    test_end_date = test_start_date + pd.DateOffset(months=6)
    
    test_data = final_price_df_1[(final_price_df_1['Date']>=test_start_date)&(final_price_df_1['Date']<=test_end_date)]
    test_data = pd.merge(test_data,weights_df,on='company',how='left')
    
    test_portfolio_return,temp_test = portfolio_generation(test_data,100000)
    test_drawdown_df = calculate_drawdown(test_portfolio_return)
    test_portfolio_return = pd.concat([test_portfolio_return,test_drawdown_df],axis=1)

    # Ratio Generation
    test_sharpe = sharpe_calculation(test_portfolio_return)
    test_calmar = calculate_calmar_ratio(test_portfolio_return)
    test_calmar.set_index('Name',inplace=True)
    test_var = calcualte_var(test_portfolio_return)
    test_drawdown_details_df = drawdown_details(test_portfolio_return)
    test_drawdown_details_df['Portfolio'] = test_drawdown_details_df['Portfolio'].str.replace('_drawdown',"")
    test_drawdown_details_df.set_index('Portfolio',inplace=True)
    metric_test = pd.concat([test_sharpe,test_calmar,test_drawdown_details_df[['Max Drawdown']],test_var],axis=1)

    metrics_for_scaling_train = metric_train.copy()
    metrics_for_scaling_test = metric_test.copy()
    
    # Flip bad metrics so lower is better → higher score
    for col in ['Max Drawdown', 'Var']:
        metrics_for_scaling_train[col] = -metrics_for_scaling_train[col]
        metrics_for_scaling_test[col] = -metrics_for_scaling_test[col]
    
    # Scale
    scaler = MinMaxScaler()
    train_scaled = pd.DataFrame(
        scaler.fit_transform(metrics_for_scaling_train),
        columns=metrics_for_scaling_train.columns,
        index=metrics_for_scaling_train.index
    )
    test_scaled = pd.DataFrame(
        scaler.transform(metrics_for_scaling_test),
        columns=metrics_for_scaling_test.columns,
        index=metrics_for_scaling_test.index
    )

    # Weighted score
    weights = {'Sharpe': 0.35, 'Calmar Ratio': 0.35, 'Max Drawdown': 0.1, 'Var': 0.2}
    train_scaled['score'] = train_scaled.mul(pd.Series(weights)).sum(axis=1)
    test_scaled['score'] = test_scaled.mul(pd.Series(weights)).sum(axis=1)


    # Getting Strategy with Highest Score
    scores = test_scaled['score']
    if (scores>0).any():
        best_strategy = scores.idxmax()
    else:
        best_strategy = scores.idxmax()
    strat_col = name_mapping[best_strategy]

    strategy_start_date =  test_end_date
    strategy_end_date = test_end_date + pd.DateOffset(months=6)
    strat_df = final_price_df_1[(final_price_df_1['Date']>strategy_start_date)&(final_price_df_1['Date']<=strategy_end_date)]
    strat_df = pd.merge(strat_df,weights_df[['company',strat_col]],on='company',how='left')

    if strat_df.empty:
        print("No data for strategy period starting", strategy_start_date, "---> ending loop.")
        break

    # Portfolio Generation for strat 2 amount
    if (
        'strat_df_portfolio_generation' not in locals()
        or strat_df_portfolio_generation is None
        or strat_df_portfolio_generation.empty
    ):
    
        # Safely get initial amount
        filtered = initial_strategy_portfolio_genreation[
            initial_strategy_portfolio_genreation['Date'] == 
            initial_strategy_portfolio_genreation['Date'].max()
        ]
    
        if filtered.empty:
            raise ValueError("No matching date found in initial_strategy_portfolio_genreation.")
    
        if 'position_value' not in filtered.columns:
            raise KeyError("'position_value' column not found in initial_strategy_portfolio_genreation.")
    
        initial_amt = filtered['position_value'].iloc[0]
    
        strat_df_portfolio_generation, strat_df_info = portfolio_generation(strat_df, initial_amt)
        strat_df_portfolio_generation.columns = ['Date'] + renamed_cols
        print('Amount Invested ------------->', initial_amt)
    
    else:
        # Existing data branch
        filtered = strat_df_portfolio_generation[
            strat_df_portfolio_generation['Date'] == strat_df_portfolio_generation['Date'].max()]
    
        if filtered.empty:
            raise ValueError("No matching date found in strat_df_portfolio_generation.")
    
        if 'position_value' not in filtered.columns:
            raise KeyError("'position_value' column not found in strat_df_portfolio_generation.")
    
        amt = filtered['position_value'].iloc[0]
        strat_df_portfolio_generation, strat_df_info = portfolio_generation(strat_df, amt)
        col = [x for x in strat_df_portfolio_generation.columns if 'position_value' in x][0]
        col_str = col.split('_')[0]
        strat_df_portfolio_generation['strategy_used'] = col_str
        strat_df_portfolio_generation.columns = ['Date'] + renamed_cols + ['strategy_used']
        strategies.append(col_str)
        print('Amount Invested ------------->', amt)
    display(strat_df_portfolio_generation)


    final_df = pd.concat([final_df,strat_df_portfolio_generation])
    
    start_date = start_date + pd.DateOffset(months=6)
    # end_date = end_date + pd.DateOffset(months=6)

    
    
        
    
        
        


2022-06-01 00:00:00
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
Amount Invested -------------> 125573.82240814803


,Date,position_value,daily_return,rolling_volatility_21D,cummulative_return
0,2023-06-05,125573.571261,0.000000,NaN,0.000000
1,2023-06-06,126293.820263,0.005736,NaN,0.005736
2,2023-06-07,127880.974771,0.012567,NaN,0.018375
3,2023-06-08,127273.537199,-0.004750,NaN,0.013538
4,2023-06-09,127103.711117,-0.001334,NaN,0.012185
...,...,...,...,...,...
118,2023-11-24,151630.412059,-0.000926,0.005371,0.207503
119,2023-11-28,152223.479157,0.003911,0.004024,0.212225
120,2023-11-29,152832.073576,0.003998,0.003832,0.217072
121,2023-11-30,153871.221307,0.006799,0.003876,0.225347


2022-12-01 00:00:00
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
Amount Invested -------------> 155076.299466051


,Date,position_value,daily_return,rolling_volatility_21D,cummulative_return,strategy_used
0,2023-12-04,155082.502518,0.000000,NaN,0.000000,Equal
1,2023-12-05,156813.866312,0.011164,NaN,0.011164,Equal
2,2023-12-06,157798.059921,0.006276,NaN,0.017510,Equal
3,2023-12-07,158361.701803,0.003572,NaN,0.021145,Equal
4,2023-12-08,157930.884077,-0.002720,NaN,0.018367,Equal
...,...,...,...,...,...,...
115,2024-05-27,200801.950885,0.000319,0.008283,0.294807,Equal
116,2024-05-28,199563.565182,-0.006167,0.008487,0.286822,Equal
117,2024-05-29,198418.737165,-0.005737,0.008670,0.279440,Equal
118,2024-05-30,196188.153953,-0.011242,0.009103,0.265057,Equal


2023-06-01 00:00:00
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
Amount Invested -------------> 196959.3033872038


...

2023-12-01 00:00:00
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
Amount Invested -------------> 201670.4688623441


,Date,position_value,daily_return,rolling_volatility_21D,cummulative_return,strategy_used
0,2024-12-03,201670.267192,0.000000,NaN,0.000000,MeanVar
1,2024-12-04,201547.718069,-0.000608,NaN,-0.000608,MeanVar
2,2024-12-05,203093.376173,0.007669,NaN,0.007057,MeanVar
3,2024-12-06,203315.022384,0.001091,NaN,0.008156,MeanVar
4,2024-12-09,202940.065600,-0.001844,NaN,0.006296,MeanVar
...,...,...,...,...,...,...
118,2025-05-27,201164.136162,-0.004329,0.010206,-0.002510,MeanVar
119,2025-05-28,200404.086153,-0.003778,0.010106,-0.006278,MeanVar
120,2025-05-29,200915.741033,0.002553,0.010073,-0.003741,MeanVar
121,2025-05-30,199571.584520,-0.006690,0.010199,-0.010407,MeanVar


2024-06-01 00:00:00
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
Amount Invested -------------> 199017.3382655726


,Date,position_value,daily_return,rolling_volatility_21D,cummulative_return,strategy_used
0,2025-06-03,199017.537283,0.000000,NaN,0.000000,RiskParity
1,2025-06-04,199332.135534,0.001581,NaN,0.001581,RiskParity
2,2025-06-05,199950.234713,0.003101,NaN,0.004687,RiskParity
3,2025-06-06,201989.345041,0.010198,NaN,0.014932,RiskParity
4,2025-06-09,204086.059183,0.010380,NaN,0.025468,RiskParity
5,2025-06-10,204513.965596,0.002097,NaN,0.027618,RiskParity
6,2025-06-11,204253.402420,-0.001274,NaN,0.026309,RiskParity
7,2025-06-12,201165.792556,-0.015117,NaN,0.010794,RiskParity
8,2025-06-13,199910.722959,-0.006239,NaN,0.004488,RiskParity
9,2025-06-16,201582.442909,0.008362,NaN,0.012888,RiskParity


2024-12-01 00:00:00
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return
No data for strategy period starting 2025-12-02 00:00:00 ---> ending loop.


In [123]:
final_df
df_strategy = final_df[['Date','position_value','strategy_used']]
df_strategy['daily_return'] = df_strategy['position_value'].pct_change().fillna(0)
df_strategy['cummulative_return'] = (1 + df_strategy['daily_return']).cumprod() - 1

In [124]:
df_strategy['strategy_used'].value_counts()

strategy_used
HRP           250
Equal         245
MeanVar       123
RiskParity     42
Name: count, dtype: int64

In [125]:
df_strategy

,Date,position_value,strategy_used,daily_return,cummulative_return
0,2022-06-01,100000.000000,HRP,0.000000,0.000000
1,2022-06-02,100314.304169,HRP,0.003143,0.003143
2,2022-06-03,99227.940842,HRP,-0.010830,-0.007721
3,2022-06-06,98998.733907,HRP,-0.002310,-0.010013
4,2022-06-07,98233.286611,HRP,-0.007732,-0.017667
...,...,...,...,...,...
37,2025-07-24,203870.078215,RiskParity,-0.004744,1.038701
38,2025-07-25,201144.832143,RiskParity,-0.013368,1.011448
39,2025-07-28,200491.147143,RiskParity,-0.003250,1.004911
40,2025-07-29,202097.038258,RiskParity,0.008010,1.020970


In [126]:
import pandas as pd
import numpy as np

def strategy_performance_report(df_strategy, var_level=0.05):
    """
    Calculate key performance metrics for a trading strategy DataFrame.
    Required columns: ['Date', 'position_value', 'strategy_used', 'daily_return']
    
    var_level: float -> e.g. 0.05 means 5% VaR
    """

    # Ensure data is sorted by Date
    df_strategy = df_strategy.sort_values('Date').reset_index(drop=True)

    # 1. CAGR
    initial_val = df_strategy['position_value'].iloc[0]
    final_val = df_strategy['position_value'].iloc[-1]
    num_days = df_strategy.shape[0]
    CAGR = (final_val / initial_val) ** (252 / num_days) - 1

    # 2. Sharpe Ratio
    daily_mean = df_strategy['daily_return'].mean()
    daily_std = df_strategy['daily_return'].std()
    sharpe = (daily_mean * 252) / (daily_std * np.sqrt(252)) if daily_std != 0 else np.nan

    # 3. Max Drawdown
    cum_max = df_strategy['position_value'].cummax()
    drawdown = df_strategy['position_value'] / cum_max - 1
    max_drawdown = drawdown.min()

    # 4. Calmar Ratio
    calmar = CAGR / abs(max_drawdown) if max_drawdown != 0 else np.nan

    # 5. Volatility (Annualized)
    volatility = daily_std * np.sqrt(252)

    # 6. Win Rate
    win_rate = (df_strategy['daily_return'] > 0).mean()

    # 7. Value at Risk (Historical)
    var_5 = np.percentile(df_strategy['daily_return'], var_level * 100)

    # 8. Strategy usage distribution
    strategy_counts = df_strategy['strategy_used'].value_counts(normalize=True) * 100

    # Create output dictionary
    metrics = {
        'CAGR': CAGR,
        'Sharpe Ratio': sharpe,
        'Max Drawdown': max_drawdown,
        'Calmar Ratio': calmar,
        'Volatility': volatility,
        'Win Rate': win_rate,
        f'{int(var_level*100)}% VaR': var_5
    }

    # Print metrics
    print("\n===== Strategy Performance Report =====")
    for k, v in metrics.items():
        print(f"{k:15}: {v:.4f}")
    
    print("\n===== Strategy Usage (%) =====")
    print(strategy_counts.round(2))

    return metrics, strategy_counts

# Example usage:
# metrics, strategy_counts = strategy_performance_report(df_strategy)


In [127]:
metrics,count = strategy_performance_report(df_strategy)


===== Strategy Performance Report =====
CAGR           : 0.2543
Sharpe Ratio   : 1.7574
Max Drawdown   : -0.1977
Calmar Ratio   : 1.2862
Volatility     : 0.1341
Win Rate       : 0.5875
5% VaR         : -0.0130

===== Strategy Usage (%) =====
strategy_used
HRP           37.88
Equal         37.12
MeanVar       18.64
RiskParity     6.36
Name: proportion, dtype: float64


In [128]:
pd.DataFrame([metrics]).T

,0
CAGR,0.254265
Sharpe Ratio,1.757372
Max Drawdown,-0.197689
Calmar Ratio,1.286188
Volatility,0.134106
Win Rate,0.587484
5% VaR,-0.013033


In [129]:
final_price_df_weights = pd.merge(final_price_df,
                                 weights_df,
                                 on='company',
                                 how='left')

In [130]:
final_price_df_weights = final_price_df_weights[(final_price_df_weights['Date']>='2022-06-01')]

In [131]:
weights_portfolio_returns,overall_df = portfolio_generation(final_price_df_weights,100000)

In [132]:
weights_portfolio_returns[['Date']+[x for x in weights_portfolio_returns.columns if 'cummulative_return' in x]]

,Date,HRP_cummulative_return,MeanVar_cummulative_return,Equal_cummulative_return,RiskParity_cummulative_return
0,2022-06-01,0.000000,0.000000,0.000000,0.000000
1,2022-06-02,0.002632,0.002082,0.003622,-0.000223
2,2022-06-03,-0.008857,-0.005008,-0.007939,-0.012672
3,2022-06-06,-0.011447,-0.009506,-0.010069,-0.013118
4,2022-06-07,-0.019553,-0.028150,-0.016927,-0.021804
...,...,...,...,...,...
778,2025-07-24,0.954804,0.563431,1.182102,0.916393
779,2025-07-25,0.935441,0.550708,1.153715,0.895122
780,2025-07-28,0.927300,0.539294,1.139544,0.884985
781,2025-07-29,0.941515,0.544546,1.157580,0.900859


In [133]:
sharpe_overall = sharpe_calculation(weights_portfolio_returns)
# calmar_overall = calculate_calmar_ratio(weights_portfolio_returns)

In [134]:
drawdown_overall = calculate_drawdown(weights_portfolio_returns)
weights_portfolio_returns = pd.concat([weights_portfolio_returns,drawdown_overall],axis=1)
calmar_overall = calculate_calmar_ratio(weights_portfolio_returns)



HRP_cummulative_return
MeanVar_cummulative_return
Equal_cummulative_return
RiskParity_cummulative_return


In [135]:
calmar_overall

,Name,Calmar Ratio
0,HRP,1.194528
1,MeanVar,1.067731
2,Equal,1.301650
3,RiskParity,1.065168


### Volatility

In [136]:
lst = []
for i in [x for x in weights_portfolio_returns.columns if 'rolling_volatility_21D' in x]:
    name = i.replace('_rolling_volatility_21D',"")
    for j in [x for x in weights_portfolio_returns.columns if 'daily_return' in x]:
        if name in j:
            daily_std = weights_portfolio_returns[j].std()
            volatility = daily_std * np.sqrt(252)
            lst.append({
                'Name':name,
                'Volatility':volatility
            })
            
    

In [137]:
volatility_overall = pd.DataFrame(lst)

In [138]:
strategy_ratios = pd.DataFrame([metrics]).T

In [139]:
volatility_overall,sharpe_overall,calmar_overall

(         Name  Volatility
 0         HRP    0.139151
 1     MeanVar    0.111445
 2       Equal    0.163565
 3  RiskParity    0.144345,
               Sharpe
 HRP         1.721387
 MeanVar     1.374904
 Equal       1.731142
 RiskParity  1.612024,
          Name  Calmar Ratio
 0         HRP      1.194528
 1     MeanVar      1.067731
 2       Equal      1.301650
 3  RiskParity      1.065168)

### CAGR

In [140]:
lst = []
weights_portfolio_returns = weights_portfolio_returns.sort_values(by='Date')

for i in [x for x in weights_portfolio_returns.columns if '_position_value' in x]:
    name = i.replace('_position_value', "")
    num_days = weights_portfolio_returns.shape[0]

    initial_value = weights_portfolio_returns.loc[weights_portfolio_returns['Date'] == weights_portfolio_returns['Date'].min(), i].iloc[0]
    final_value = weights_portfolio_returns.loc[weights_portfolio_returns['Date'] == weights_portfolio_returns['Date'].max(), i].iloc[0]

    cagr = (final_value / initial_value) ** (252 / num_days) - 1

    lst.append({
        'Name': name,
        'CAGR': cagr
    })

cagr_overall = pd.DataFrame(lst)


In [146]:
overall_drawdown = drawdown_details(weights_portfolio_returns)
overall_drawdown = overall_drawdown[['Portfolio','Max Drawdown']]

overall_drawdown['Portfolio'] = overall_drawdown['Portfolio'].str.replace('_drawdown','')
overall_drawdown.rename(columns={'Portfolio':'Name'},inplace=True)

var_overall = calcualte_var(weights_portfolio_returns)
var_overall = var_overall.reset_index()
var_overall.rename(columns={'index':'Name'},inplace=True)

### Win Rate

In [151]:
lst = []
for i in [x for x in weights_portfolio_returns.columns if '_daily_return' in x]:
    name = i.replace('_daily_return',"")
    df = weights_portfolio_returns[['Date',i]]
    # display(df)
    win_rate = (df[i] > 0).mean()
    lst.append({
        'Name':name,
        'Win Rate':win_rate
    })
winrate_overall = pd.DataFrame(lst)    

In [153]:
strategy_ratios.rename(columns={0:'Strategy'},inplace=True)

In [154]:
strategy_ratios.T

,CAGR,Sharpe Ratio,Max Drawdown,Calmar Ratio,Volatility,Win Rate,5% VaR
Strategy,0.254265,1.757372,-0.197689,1.286188,0.134106,0.587484,-0.013033


#### Concating

In [155]:
cagr_overall = cagr_overall.set_index('Name')
cagr_overall.index = ['HRP', 'MeanVar', 'Equal', 'RiskParity']
sharpe_overall = sharpe_overall
overall_drawdown = overall_drawdown.set_index('Name')
calmar_overall = calmar_overall.set_index('Name')
volatility_overall = volatility_overall.set_index('Name')
winrate_overall = winrate_overall.set_index('Name')
var_overall = var_overall.set_index('Name')

In [156]:
ratios = pd.concat([cagr_overall,sharpe_overall,overall_drawdown,calmar_overall,volatility_overall,winrate_overall,var_overall],axis=1).T
ratios.index = strategy_ratios.index

In [157]:
comparision_df = pd.concat([strategy_ratios,ratios],axis=1)

In [158]:
ratios


,HRP,MeanVar,Equal,RiskParity
CAGR,0.238256,0.151257,0.280741,0.230409
Sharpe Ratio,1.721387,1.374904,1.731142,1.612024
Max Drawdown,-0.209563,-0.148361,-0.229006,-0.228432
Calmar Ratio,1.194528,1.067731,1.301650,1.065168
Volatility,0.139151,0.111445,0.163565,0.144345
Win Rate,0.591315,0.546616,0.602810,0.588761
5% VaR,-0.013740,-0.011525,-0.016313,-0.014331


In [159]:
final_df['strategy_used'].value_counts()

strategy_used
HRP           250
Equal         245
MeanVar       123
RiskParity     42
Name: count, dtype: int64

In [178]:
strategy_ratios

,Strategy
CAGR,0.254265
Sharpe Ratio,1.757372
Max Drawdown,-0.197689
Calmar Ratio,1.286188
Volatility,0.134106
Win Rate,0.587484
5% VaR,-0.013033


In [179]:
ratios

,HRP,MeanVar,Equal,RiskParity
CAGR,0.238256,0.151257,0.280741,0.230409
Sharpe Ratio,1.721387,1.374904,1.731142,1.612024
Max Drawdown,-0.209563,-0.148361,-0.229006,-0.228432
Calmar Ratio,1.194528,1.067731,1.301650,1.065168
Volatility,0.139151,0.111445,0.163565,0.144345
Win Rate,0.591315,0.546616,0.602810,0.588761
5% VaR,-0.013740,-0.011525,-0.016313,-0.014331


#### Get Nifty Data

In [162]:
def get_nifty_data(start_date,end_date):
    
    price = yf.download(tickers='^NSEI',start=start_date,end=end_date)
    price.columns = [f"{ticker.split('.')[0]}_{typ}" for typ, ticker in price.columns]
    price = price.reset_index()
    
    nifty_df = price[['Date','^NSEI_Close']]
    
    nifty_df['daily_return'] = nifty_df['^NSEI_Close'].pct_change()
    nifty_df['daily_return'] = nifty_df['daily_return'].fillna(0)
    nifty_df['index_cummulative_return'] = (1+ nifty_df['daily_return']).cumprod() -1
    
    
    investment_amount = 100000
    nifty_df['index_value'] = (1+nifty_df['index_cummulative_return'])*investment_amount
    nifty_df['rolling_volatility_21D'] = nifty_df['daily_return'].rolling(window=21).std()
    return nifty_df
    

In [163]:
nifty_df = get_nifty_data('2022-06-01','2025-07-31')

[*********************100%***********************]  1 of 1 completed


In [164]:
comaprision_strategy = pd.merge(df_strategy,
                               weights_portfolio_returns,
                               on='Date',
                               how='left')

In [165]:
comaprision_strategy

,Date,position_value,strategy_used,daily_return,cummulative_return,HRP_position_value,MeanVar_Opt_position_value,Equal_Weight_position_value,RiskParity_position_value,HRP_daily_return,HRP_rolling_volatility_21D,HRP_cummulative_return,MeanVar_daily_return,MeanVar_rolling_volatility_21D,MeanVar_cummulative_return,Equal_daily_return,Equal_rolling_volatility_21D,Equal_cummulative_return,RiskParity_daily_return,RiskParity_rolling_volatility_21D,RiskParity_cummulative_return,HRP_drawdown,MeanVar_drawdown,Equal_drawdown,RiskParity_drawdown
0,2022-06-01,100000.000000,HRP,0.000000,0.000000,99999.900000,100000.100000,100004.000000,100000.500000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
1,2022-06-02,100314.304169,HRP,0.003143,0.003143,100263.066791,100208.284671,100366.210071,99978.213464,0.002632,NaN,0.002632,0.002082,NaN,0.002082,0.003622,NaN,0.003622,-0.000223,NaN,-0.000223,0.000000,0.000000,0.000000,-0.000223
2,2022-06-03,99227.940842,HRP,-0.010830,-0.007721,99114.228287,99499.286129,99210.116967,98733.250552,-0.011458,NaN,-0.008857,-0.007075,NaN,-0.005008,-0.011519,NaN,-0.007939,-0.012452,NaN,-0.012672,-0.011458,-0.007075,-0.011519,-0.012672
3,2022-06-06,98998.733907,HRP,-0.002310,-0.010013,98855.247815,99049.521325,98997.082384,98688.700145,-0.002613,NaN,-0.011447,-0.004520,NaN,-0.009506,-0.002147,NaN,-0.010069,-0.000451,NaN,-0.013118,-0.014041,-0.011564,-0.013641,-0.013118
4,2022-06-07,98233.286611,HRP,-0.007732,-0.017667,98044.561514,97185.143924,98311.279334,97820.134514,-0.008201,NaN,-0.019553,-0.018823,NaN,-0.028150,-0.006928,NaN,-0.016927,-0.008801,NaN,-0.021804,-0.022127,-0.030169,-0.020474,-0.021804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2025-07-24,203870.078215,RiskParity,-0.004744,1.038701,195480.169004,156343.209447,218218.949839,191640.239582,-0.003959,0.004196,0.954804,-0.007922,0.004586,0.563431,-0.002719,0.004232,1.182102,-0.004417,0.004584,0.916393,-0.076347,-0.015099,-0.094691,-0.109120
779,2025-07-25,201144.832143,RiskParity,-0.013368,1.011448,193543.883183,155070.980653,215380.070617,189513.112099,-0.009905,0.004165,0.935441,-0.008137,0.004092,0.550708,-0.013009,0.004544,1.153715,-0.011100,0.004797,0.895122,-0.085496,-0.023113,-0.106468,-0.119008
780,2025-07-28,200491.147143,RiskParity,-0.003250,1.004911,192729.849733,153929.588438,213962.949949,188499.431883,-0.004206,0.004019,0.927300,-0.007360,0.004262,0.539294,-0.006580,0.004548,1.139544,-0.005349,0.004703,0.884985,-0.089342,-0.030304,-0.112348,-0.123720
781,2025-07-29,202097.038258,RiskParity,0.008010,1.020970,194151.339968,154454.707141,215766.623207,190086.873281,0.007376,0.004417,0.941515,0.003411,0.004383,0.544546,0.008430,0.004969,1.157580,0.008421,0.005164,0.900859,-0.082626,-0.026996,-0.104865,-0.116341


In [166]:
comaprision_strategy

,Date,position_value,strategy_used,daily_return,cummulative_return,HRP_position_value,MeanVar_Opt_position_value,Equal_Weight_position_value,RiskParity_position_value,HRP_daily_return,HRP_rolling_volatility_21D,HRP_cummulative_return,MeanVar_daily_return,MeanVar_rolling_volatility_21D,MeanVar_cummulative_return,Equal_daily_return,Equal_rolling_volatility_21D,Equal_cummulative_return,RiskParity_daily_return,RiskParity_rolling_volatility_21D,RiskParity_cummulative_return,HRP_drawdown,MeanVar_drawdown,Equal_drawdown,RiskParity_drawdown
0,2022-06-01,100000.000000,HRP,0.000000,0.000000,99999.900000,100000.100000,100004.000000,100000.500000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
1,2022-06-02,100314.304169,HRP,0.003143,0.003143,100263.066791,100208.284671,100366.210071,99978.213464,0.002632,NaN,0.002632,0.002082,NaN,0.002082,0.003622,NaN,0.003622,-0.000223,NaN,-0.000223,0.000000,0.000000,0.000000,-0.000223
2,2022-06-03,99227.940842,HRP,-0.010830,-0.007721,99114.228287,99499.286129,99210.116967,98733.250552,-0.011458,NaN,-0.008857,-0.007075,NaN,-0.005008,-0.011519,NaN,-0.007939,-0.012452,NaN,-0.012672,-0.011458,-0.007075,-0.011519,-0.012672
3,2022-06-06,98998.733907,HRP,-0.002310,-0.010013,98855.247815,99049.521325,98997.082384,98688.700145,-0.002613,NaN,-0.011447,-0.004520,NaN,-0.009506,-0.002147,NaN,-0.010069,-0.000451,NaN,-0.013118,-0.014041,-0.011564,-0.013641,-0.013118
4,2022-06-07,98233.286611,HRP,-0.007732,-0.017667,98044.561514,97185.143924,98311.279334,97820.134514,-0.008201,NaN,-0.019553,-0.018823,NaN,-0.028150,-0.006928,NaN,-0.016927,-0.008801,NaN,-0.021804,-0.022127,-0.030169,-0.020474,-0.021804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2025-07-24,203870.078215,RiskParity,-0.004744,1.038701,195480.169004,156343.209447,218218.949839,191640.239582,-0.003959,0.004196,0.954804,-0.007922,0.004586,0.563431,-0.002719,0.004232,1.182102,-0.004417,0.004584,0.916393,-0.076347,-0.015099,-0.094691,-0.109120
779,2025-07-25,201144.832143,RiskParity,-0.013368,1.011448,193543.883183,155070.980653,215380.070617,189513.112099,-0.009905,0.004165,0.935441,-0.008137,0.004092,0.550708,-0.013009,0.004544,1.153715,-0.011100,0.004797,0.895122,-0.085496,-0.023113,-0.106468,-0.119008
780,2025-07-28,200491.147143,RiskParity,-0.003250,1.004911,192729.849733,153929.588438,213962.949949,188499.431883,-0.004206,0.004019,0.927300,-0.007360,0.004262,0.539294,-0.006580,0.004548,1.139544,-0.005349,0.004703,0.884985,-0.089342,-0.030304,-0.112348,-0.123720
781,2025-07-29,202097.038258,RiskParity,0.008010,1.020970,194151.339968,154454.707141,215766.623207,190086.873281,0.007376,0.004417,0.941515,0.003411,0.004383,0.544546,0.008430,0.004969,1.157580,0.008421,0.005164,0.900859,-0.082626,-0.026996,-0.104865,-0.116341


In [167]:
def plot_data(nifty,df):
    
    highlight_date = df['Date'].max()
    highlight_date = pd.to_datetime(highlight_date,format="%Y-%m-%d")
    index_return = f"{round(nifty[nifty['Date'] == highlight_date]['index_cummulative_return'].iloc[0] * 100, 2)}%"
    hrp_return = f"{round(df[df['Date'] == highlight_date]['HRP_cummulative_return'].iloc[0] * 100, 2)}%"
    meanvar_return = f"{round(df[df['Date'] == highlight_date]['MeanVar_cummulative_return'].iloc[0] * 100, 2)}%"
    equaldaily_return = f"{round(df[df['Date'] == highlight_date]['Equal_cummulative_return'].iloc[0] * 100, 2)}%"
    riskparity_return = f"{round(df[df['Date'] == highlight_date]['RiskParity_cummulative_return'].iloc[0] * 100, 2)}%"
    strategy_return = f"{round(df[df['Date'] == highlight_date]['cummulative_return'].iloc[0] * 100, 2)}%"
    
    index_return_val = nifty[nifty['Date'] == highlight_date]['index_cummulative_return'].iloc[0]
    hrp_return_val = df[df['Date'] == highlight_date]['HRP_cummulative_return'].iloc[0]
    meanvar_return_val = df[df['Date'] == highlight_date]['MeanVar_cummulative_return'].iloc[0]
    equaldaily_return_val = df[df['Date'] == highlight_date]['Equal_cummulative_return'].iloc[0]
    riskparity_return_val = df[df['Date'] == highlight_date]['RiskParity_cummulative_return'].iloc[0]
    strategy_return_val = df[df['Date'] == highlight_date]['cummulative_return'].iloc[0]
    
    plt.figure(figsize=(16,8))
    
    sns.lineplot(data=nifty,
                x='Date',
                y='index_cummulative_return',
                color='black',
                linestyle= '--',
                label='Nifty Return')
    
    sns.lineplot(data=df,
                x='Date',
                y='HRP_cummulative_return',
                color='blue',
                label='HRP Return')
    
    sns.lineplot(data=df,
                x='Date',
                y='MeanVar_cummulative_return',
                color='red',
                label='Mean Var Opt. Return')
    
    sns.lineplot(data=df,
                x='Date',
                y='Equal_cummulative_return',
                color='green',
                label='Equal Weight Return')
    
    sns.lineplot(data=df,
                x='Date',
                y='RiskParity_cummulative_return',
                color='#17BECF',
                label='Risk Parity Return')
    
    sns.lineplot(data=df,
                x='Date',
                y='cummulative_return',
                color='#FFD700',
                label='Meta Strategy Return')
    
    plt.axhline(y=index_return_val, color='black', linestyle=':', linewidth=1)
    plt.axhline(y=hrp_return_val, color='blue', linestyle=':', linewidth=1)
    plt.axhline(y=meanvar_return_val, color='red', linestyle=':', linewidth=1)
    plt.axhline(y=equaldaily_return_val, color='green', linestyle=':', linewidth=1)
    plt.axhline(y=riskparity_return_val, color='#17BECF', linestyle=':', linewidth=1)
    plt.axhline(y=strategy_return_val, color='#FFD700', linestyle=':', linewidth=1)
    
    plt.text(highlight_date,
             index_return_val + 0.01,
             f"Nifty: {index_return}",
             color='black',
             fontsize=10,
             ha='left')
    
    plt.text(highlight_date,
             hrp_return_val + 0.01,
             f"HRP: {hrp_return}",
             color='blue',
             fontsize=10,
             ha='left')
    
    plt.text(highlight_date,
             meanvar_return_val + 0.01,
             f"Mean Var Opt.: {meanvar_return}",
             color='red',
             fontsize=10,
             ha='left')
    
    plt.text(highlight_date,
             equaldaily_return_val + 0.01,
             f"Equal Weight: {equaldaily_return}",
             color='green',
             fontsize=10,
             ha='left')

    plt.text(highlight_date,
             riskparity_return_val + 0.01,
             f"Risk Weight: {riskparity_return}",
             color='#17BECF',
             fontsize=10,
             ha='left')
    
    plt.text(highlight_date,
             strategy_return_val + 0.01,
             f"Strategy Weight: {strategy_return}",
             color='#FFD700',
             fontsize=10,
             ha='left')
    
    # plt.title('Nifty Index Vs Heirchal Risk Parity Return Vs Markowitz Optimizer Vs Equal Weighted Vs Risk Parity on (1L Invested)')
    plt.title('Optimizers Comparision (Test Data)')
    plt.xlabel('Date')
    plt.ylabel('Returns')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    # plt.show()
    

In [180]:
plot_data(nifty_df,comaprision_strategy)
plt.savefig('latest_data.png')
plt.close()

In [171]:
import matplotlib.pyplot as plt
import seaborn as sns

# Use a clean Seaborn style
sns.set_theme(style="whitegrid")

# Create color palette
palette = sns.color_palette("muted", 4)


plt.figure(figsize=(12,6))


for idx, col in enumerate(['HRP_drawdown', 'MeanVar_drawdown', 'Equal_drawdown', 'RiskParity_drawdown']):
    plt.plot(
        portfolio_return_train['Date'], 
        portfolio_return_train[col], 
        label=col, 
        color=palette[idx], 
        linewidth=1.8
    )
    
    
    min_dd = portfolio_return_train[col].min()
    min_date = portfolio_return_train.loc[portfolio_return_train[col].idxmin(), 'Date']
    plt.annotate(
        f"Max DD: {min_dd:.2%}",
        xy=(min_date, min_dd),
        xytext=(min_date, min_dd - 0.02),
        arrowprops=dict(facecolor=palette[idx], arrowstyle="->", lw=1),
        fontsize=9,
        color=palette[idx]
    )


plt.title('Train Portfolio Drawdowns Over Time', fontsize=14, weight='bold')
plt.ylabel('Drawdown')
plt.xlabel('Date')
plt.axhline(0, color='black', linewidth=1)
plt.legend()
plt.tight_layout()

plt.savefig('Train Portfolio Drawdown.png')
plt.close()
plt.show()



NameError: name 'portfolio_return_train' is not defined

<Figure size 1200x600 with 0 Axes>

In [172]:
strat_drawdoewn = calculate_drawdown(df_strategy)

cummulative_return


In [173]:
df_strategy = pd.concat([df_strategy,strat_drawdoewn],axis=1)

In [174]:
df_strategy

,Date,position_value,strategy_used,daily_return,cummulative_return,cummulative_drawdown
0,2022-06-01,100000.000000,HRP,0.000000,0.000000,0.000000
1,2022-06-02,100314.304169,HRP,0.003143,0.003143,0.000000
2,2022-06-03,99227.940842,HRP,-0.010830,-0.007721,-0.010830
3,2022-06-06,98998.733907,HRP,-0.002310,-0.010013,-0.013114
4,2022-06-07,98233.286611,HRP,-0.007732,-0.017667,-0.020745
...,...,...,...,...,...,...
37,2025-07-24,203870.078215,RiskParity,-0.004744,1.038701,-0.085427
38,2025-07-25,201144.832143,RiskParity,-0.013368,1.011448,-0.097653
39,2025-07-28,200491.147143,RiskParity,-0.003250,1.004911,-0.100585
40,2025-07-29,202097.038258,RiskParity,0.008010,1.020970,-0.093381


In [182]:
comparision_df.to_csv('ratios.csv')